In [ ]:
!pip install transformers
!pip install keras
!pip install Sentencepiece
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install datasets
!pip install -U scikit-learn

In [1]:
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["WANDB_DISABLED"] = "true"

In [2]:
import transformers
import torch
#import keras
from datasets import load_dataset, concatenate_datasets, Dataset

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from functions import *

from transformers import TextClassificationPipeline
from transformers.pipelines.pt_utils import KeyDataset

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer

2022-12-15 12:02:15.272779: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# Training of Baseline Classifiers

In [4]:
#features = ["valence"]
label_num = 4
#conf_scores = [0.5, 0.7, 0.95]
conf_score = 0.5
#load_tokenizer("distilbert-base-german-cased")

In [14]:
train = load_tokenize_enhanced_set("../Ablation_Studies/forum/sublabel/train_sublabel_preproc.csv")
test = load_tokenize_enhanced_set("../Ablation_Studies/forum/sublabel/test_sublabel_preproc.csv")
training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")

Using custom data configuration default-3edc0ad1ac61e499
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-3edc0ad1ac61e499/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?ba/s]

Using custom data configuration default-4bcdf5d906130417
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-4bcdf5d906130417/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?ba/s]

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [15]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
trainer = Trainer(model=model, args=training_args, train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics)

loading configuration file config.json from cache at /home/jovyan/.cache/huggingface/hub/models--bert-base-german-cased/snapshots/702774c02b32a4f360d5fea60ab034d64bf0141c/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-german-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 300

In [16]:
trainer.train()
trainer.save_model("classif/final_sublabel_model")
tokenizer.save_pretrained("classif/final_sublabel_model")

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: function_Sentence, nonfunction_Sentence, non_noun_verb_adj, Sentence, non_noun, non_verb, non_verb_aux, non_noun_adj. If function_Sentence, nonfunction_Sentence, non_noun_verb_adj, Sentence, non_noun, non_verb, non_verb_aux, non_noun_adj are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2411
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps 

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Mcc
1,No log,0.546480,0.788591,0.733440,0.696260,0.783548,0.597685
2,No log,0.519716,0.818792,0.738956,0.742000,0.737157,0.623796
3,No log,0.654404,0.830537,0.761279,0.755587,0.768865,0.650737


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: function_Sentence, nonfunction_Sentence, non_noun_verb_adj, Sentence, non_noun, non_verb, non_verb_aux, non_noun_adj. If function_Sentence, nonfunction_Sentence, non_noun_verb_adj, Sentence, non_noun, non_verb, non_verb_aux, non_noun_adj are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 596
  Batch size = 16
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: function_Sentence, nonfunctio

('classif/final_sublabel_model/tokenizer_config.json',
 'classif/final_sublabel_model/special_tokens_map.json',
 'classif/final_sublabel_model/vocab.txt',
 'classif/final_sublabel_model/added_tokens.json',
 'classif/final_sublabel_model/tokenizer.json')

# Finetuning with winning confidence thresholds

#### Concatenating GLoHBCD-Weight, GLoHBCD-Smoke + Annomi

In [5]:
weight_loss = pd.read_csv("new_data_both.csv")
weight_loss = weight_loss["split"].copy()
train_rest = pd.read_csv("Nonsmoking_addition/baldNichtraucher_annotiert.csv", sep=";")
valence_ns = pd.read_csv("Nonsmoking_addition/ns_val_train_set.csv")
valence_ns = valence_ns["Sentence"].copy()
annomi = pd.read_csv("Nonsmoking_addition/val_AnnoMI_predicted_set.csv")
annomi = annomi["Sentence"].copy()

labels = ["R", "C", "TS"]

train_rest = train_rest[~train_rest["label"].isin(labels)].copy()
train_rest = train_rest["sentence"].copy()

df = pd.concat([weight_loss, train_rest, valence_ns, annomi])
df.to_csv("Valence_all_new_data.csv")

In [20]:
weight_loss = pd.read_csv("new_data_both.csv")
weight_loss = weight_loss["split"].copy()
train_rest = pd.read_csv("Nonsmoking_addition/baldNichtraucher_annotiert.csv", sep=";")
valence_ns = pd.read_csv("Nonsmoking_addition/ns_label_train_set.csv")
valence_ns = valence_ns["Sentence"].copy()
annomi = pd.read_csv("Nonsmoking_addition/val_AnnoMI_predicted_set.csv")
annomi = annomi["Sentence"].copy()

labels = ["R", "C", "TS"]

train_rest = train_rest[~train_rest["label"].isin(labels)].copy()
train_rest = train_rest["sentence"].copy()

df = pd.concat([weight_loss, train_rest, valence_ns, annomi])
df.to_csv("Label_all_new_data.csv")

In [21]:
weight_loss = pd.read_csv("new_data_both.csv")
weight_loss = weight_loss["split"].copy()
train_rest = pd.read_csv("Nonsmoking_addition/baldNichtraucher_annotiert.csv", sep=";")
valence_ns = pd.read_csv("Nonsmoking_addition/ns_sublabel_train_set.csv")
valence_ns = valence_ns["Sentence"].copy()
annomi = pd.read_csv("Nonsmoking_addition/val_AnnoMI_predicted_set.csv")
annomi = annomi["Sentence"].copy()

labels = ["R", "C", "TS"]

train_rest = train_rest[~train_rest["label"].isin(labels)].copy()
train_rest = train_rest["sentence"].copy()

df = pd.concat([weight_loss, train_rest, valence_ns, annomi])
df.to_csv("Sublabel_all_new_data.csv")

### creating pseudo labels

In [9]:
# Valence
label_num = 2
conf_score = 0.5
val_FN_winner = load_dataset("csv", data_files="Valence_all_new_data.csv", split="train")

model = AutoModelForSequenceClassification.from_pretrained("classif/final_valence_model", num_labels=label_num)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

label_0_scores = []
label_1_scores = []
#for out in pipe(list(X_train_base[i]), batch_size=8, truncation="only_first"):
for out in pipe(KeyDataset(val_FN_winner, "0"), batch_size=8, truncation="only_first"): 
    label_0_scores.append(out[0]['score'])
    label_1_scores.append(out[1]['score'])
pred_train_df = pd.DataFrame({"Sentence": val_FN_winner[:]["0"], "0":label_0_scores, "1": label_1_scores})
_1 = pred_train_df[pred_train_df["0"] > conf_score].copy()
a_1 = pred_train_df[pred_train_df["1"] > conf_score].copy()
_1["labels"] = 0
a_1["labels"] = 1
predict_1 = pd.concat([_1,a_1]).sample(frac=0.2)
#predict_1.rename(columns={'split':'Sentence'}, inplace=True)
predict_1.to_csv("valence_predicted_enhanced_set.csv")

Using custom data configuration default-9a84bf4b2af12b2f
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-9a84bf4b2af12b2f/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [ ]:
label_num = 2
enhanced = load_tokenize_enhanced_set("valence_predicted_enhanced_set.csv")
orig = load_tokenize_enhanced_set("../Ablation_Studies/forum/valence/train_valence_preproc.csv")
train = concatenate_datasets([enhanced, orig]).shuffle()
test = load_tokenize_enhanced_set("../Ablation_Studies/forum/valence/test_valence_preproc.csv")

training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained("classif/final_valence_model", num_labels=label_num)
trainer = Trainer(model=model, args=training_args, train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics)

print("original:")
print(trainer.evaluate())

model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
trainer = Trainer(model=model, args=training_args, train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics)
trainer.train()

print("with automatically annotated data:")
print(trainer.evaluate())
            
trainer.save_model("classif/final_valence_fuzzy_model_ooD")

In [5]:
label_num = 3
conf_score = 0.95
lab_FN_winner = load_dataset("csv", data_files="Label_all_new_data.csv", split="train")

model = AutoModelForSequenceClassification.from_pretrained("classif/final_label_model", num_labels=label_num)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

label_0_scores = []
label_1_scores = []
label_2_scores = []
for out in pipe(KeyDataset(lab_FN_winner, "0"), batch_size=8, truncation="only_first"):
    label_0_scores.append(out[0]['score'])
    label_1_scores.append(out[1]['score'])
    label_2_scores.append(out[2]['score'])
pred_train_df = pd.DataFrame({"split": lab_FN_winner[:]["0"], "0":label_0_scores, "1": label_1_scores, "2": label_2_scores})
_1 = pred_train_df[pred_train_df["0"] > conf_score].copy()
a_1 = pred_train_df[pred_train_df["1"] > conf_score].copy()
d_1 = pred_train_df[pred_train_df["2"] > conf_score].copy()
_1["labels"] = 0
a_1["labels"] = 1
d_1["labels"] = 2
predict_1 = pd.concat([_1,a_1, d_1]).sample(frac=1)
predict_1.rename(columns={'split':'Sentence'}, inplace=True)
predict_1.to_csv("label_predicted_enhanced_set.csv")

Using custom data configuration default-e4ed113be21c524c
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-e4ed113be21c524c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
loading configuration file classif/final_label_model/config.json
Model config BertConfig {
  "_name_or_path": "classif/final_label_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_typ

In [ ]:
label_num = 3
enhanced = load_tokenize_enhanced_set("label_predicted_enhanced_set.csv")
orig = load_tokenize_enhanced_set("../Ablation_Studies/forum/label/train_label_preproc.csv")
train = concatenate_datasets([enhanced, orig]).shuffle()
test = load_tokenize_enhanced_set("../Ablation_Studies/forum/label/test_label_preproc.csv")

training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained("classif/final_label_model", num_labels=label_num)
trainer = Trainer(model=model, args=training_args, train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics)

print("original:")
print(trainer.evaluate())

model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
trainer = Trainer(model=model, args=training_args, train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics)
trainer.train()

print("with automatically annotated data:")
print(trainer.evaluate())
            
trainer.save_model("classif/final_label_fuzzy_model_ooD")

#### Sublabel

In [ ]:
# classify FN first
model = AutoModelForSequenceClassification.from_pretrained("../MI_Data/Bert_Finetuning/models/FN_finetuned_gbert/model", local_files_only=True)
tokenizer = AutoTokenizer.from_pretrained("deepset/gbert-base")
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
sublab_FN_winner = load_dataset("csv", data_files="Sublabel_all_new_data.csv", split="train")

label_0_scores = []
label_1_scores = []
    
for out in pipe(KeyDataset(sublab_FN_winner, "0"), batch_size=8, truncation="only_first"):
    label_0_scores.append(out[0]['score'])
    label_1_scores.append(out[1]['score'])
pred_train_df = pd.DataFrame({"Sentence": sublab_FN_winner[:]["0"], "0":label_0_scores, "1": label_1_scores})
a_1 = pred_train_df[pred_train_df["1"] > 0.7].copy()
a_1["labels"] = 1
predict_1 = a_1["Sentence"]
predict_1.to_csv("sublabel_FN_predicted_enhanced_set.csv")

In [3]:
label_num = 4
sublab_FN_winner = load_dataset("csv", data_files="sublabel_FN_predicted_enhanced_set.csv", split="train")
conf_score = 0.5

model = AutoModelForSequenceClassification.from_pretrained("classif/final_sublabel_model", num_labels=label_num)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)

label_0_scores = []
label_1_scores = []
label_2_scores = []
label_3_scores = []
for out in pipe(KeyDataset(sublab_FN_winner, "Sentence"), batch_size=8, truncation="only_first"):
    label_0_scores.append(out[0]['score'])
    label_1_scores.append(out[1]['score'])
    label_2_scores.append(out[2]['score'])
    label_3_scores.append(out[3]['score'])
pred_train_df = pd.DataFrame({"Sentence": sublab_FN_winner[:]['Sentence'], "0":label_0_scores, "1": label_1_scores, "2": label_2_scores, "3": label_3_scores})
#_1 = pred_train_df[pred_train_df["0"] > conf_score].copy()
#a_1 = pred_train_df[pred_train_df["1"] > conf_score].copy()
#d_1 = pred_train_df[pred_train_df["2"] > conf_score].copy()
n_1 = pred_train_df[pred_train_df["3"] > conf_score].copy()
#_1["labels"] = 0
#a_1["labels"] = 1
#d_1["labels"] = 2
n_1["labels"] = 3
#predict_1 = pd.concat([_1,a_1, d_1, n_1]).sample(frac=1)
#n_1.rename(columns={'split':'Sentence'}, inplace=True)
n_1 = n_1.sample(frac=0.2)
n_1.to_csv("sublabel_predicted_enhanced_set.csv")

Using custom data configuration default-39c3c29d3fec060e
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-39c3c29d3fec060e/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
/opt/conda/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [4]:
label_num = 4
enhanced = load_tokenize_enhanced_set("sublabel_predicted_enhanced_set.csv")
orig = load_tokenize_enhanced_set("../Ablation_Studies/forum/sublabel/train_sublabel_preproc.csv")
train = concatenate_datasets([enhanced, orig]).shuffle()
test = load_tokenize_enhanced_set("../Ablation_Studies/forum/sublabel/test_sublabel_preproc.csv")

training_args = TrainingArguments("original_forum", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained("classif/final_sublabel_model", num_labels=label_num)
trainer = Trainer(model=model, args=training_args, train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics)

print("original:")
print(trainer.evaluate())

model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
trainer = Trainer(model=model, args=training_args, train_dataset=train, eval_dataset=test, compute_metrics=compute_metrics)
trainer.train()

print("with automatically annotated data:")
print(trainer.evaluate())
            
trainer.save_model("classif/final_sublabel_fuzzy_model_ooD")

Using custom data configuration default-892611426219477b


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/jovyan/.cache/huggingface/datasets/csv/default-892611426219477b/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?ba/s]

...

with automatically annotated data:


/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Saving model checkpoint to classif/final_sublabel_fuzzy_model_ooD
Configuration saved in classif/final_sublabel_fuzzy_model_ooD/config.json


{'eval_loss': 0.6893049478530884, 'eval_accuracy': 0.7986577181208053, 'eval_f1': 0.7252735074464731, 'eval_precision': 0.7154295921537301, 'eval_recall': 0.7365822261083974, 'eval_MCC': 0.591054303471214, 'eval_runtime': 4.5168, 'eval_samples_per_second': 131.952, 'eval_steps_per_second': 8.413, 'epoch': 3.0}


Model weights saved in classif/final_sublabel_fuzzy_model_ooD/pytorch_model.bin


### Using "winning" classifiers to predict on (out-of-domain) test sets

#### Valence

In [ ]:
metrics = {}
metrics[1] = {}

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/test_valence_preproc.csv", model_path="classif/final_valence_model")
metrics[1]["GloHBCD Orig - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/test_valence_preproc.csv", model_path="classif/final_valence_fuzzy_model_ooD")
metrics[1]["GloHBCD Winner - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("Nonsmoking_addition/ns_val_test_set.csv", model_path="classif/final_valence_model")
metrics[1]["Nonsmoking Orig - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("Nonsmoking_addition/ns_val_test_set.csv", model_path="classif/final_valence_fuzzy_model_ooD")
metrics[1]["Nonsmoking Winner - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/optifast_valence_preproc.csv", model_path="classif/final_valence_model")
metrics[1]["Optifast Orig - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/optifast_valence_preproc.csv", model_path="classif/final_valence_fuzzy_model_ooD")
metrics[1]["Optifast Winner - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/virtualcoach_valence_preproc.csv", model_path="classif/final_valence_model")
metrics[1]["Virtualcoach Orig - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/virtualcoach_valence_preproc.csv", model_path="classif/final_valence_fuzzy_model_ooD")
metrics[1]["Virtualcoach Winner - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/synthetic_valence_preproc.csv", model_path="classif/final_valence_model")
metrics[1]["Synthetic Orig - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/synthetic_valence_preproc.csv", model_path="classif/final_valence_fuzzy_model_ooD")
metrics[1]["Synthetic Winner - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/WoZ_valence_preproc.csv", model_path="classif/final_valence_model")
metrics[1]["WoZ Orig - Valence"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/valence/WoZ_valence_preproc.csv", model_path="classif/final_valence_fuzzy_model_ooD")
metrics[1]["WoZ Winner - Valence"] = compute_test_metrics(forum_test, None)

In [6]:
metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()}, 
                        orient='columns').transpose()
            
metrics_df.head()

accuracy  \
1 GloHBCD Orig - Valence       0.782468   
  GloHBCD Winner - Valence     0.769481   
  Nonsmoking Orig - Valence    0.718593   
  Nonsmoking Winner - Valence  0.758794   
  Optifast Orig - Valence      0.844156   

                                                                     f1  \
1 GloHBCD Orig - Valence       [0.6051080550098232, 0.8498879761015683]   
  GloHBCD Winner - Valence     [0.5831702544031311, 0.8406881077038145]   
  Nonsmoking Orig - Valence                  [0.44, 0.8120805369127516]   
  Nonsmoking Winner - Valence  [0.5471698113207547, 0.8356164383561644]   
  Optifast Orig - Valence                   [0.625, 0.9016393442622951]   

                                                               precision  \
1 GloHBCD Orig - Valence         [0.6724890829694323, 0.818705035971223]   
  GloHBCD Winner - Valence        [0.645021645021645, 0.810966810966811]   
  Nonsmoking Orig - Valence    [0.46808510638297873, 0.7960526315789473]   
  Nonsmoking Winner - Valence   [0.5471698113207547, 0.8356164383561644]   
  Optifast Orig - Valence       [0.6666666666666666, 0.8870967741935484]   

                                                                  recall  \
1 GloHBCD Orig - Valence                      [0.55, 0.8835403726708074]   
  GloHBCD Winner - Valence      [0.5321428571428571, 0.8726708074534162]   
  Nonsmoking Orig - Valence    [0.41509433962264153, 0.8287671232876712]   
  Nonsmoking Winner - Valence   [0.5471698113207547, 0.8356164383561644]   
  Optifast Orig - Valence       [0.5882352941176471, 0.9166666666666666]   

                                  support       MCC  
1 GloHBCD Orig - Valence       [280, 644]  0.461468  
  GloHBCD Winner - Valence     [280, 644]   0.42964  
  Nonsmoking Orig - Valence     [53, 146]  0.253797  
  Nonsmoking Winner - Valence   [53, 146]  0.382786  
  Optifast Orig - Valence        [17, 60]  0.528769

In [7]:
metrics_df.to_csv("metrics/Valence_adding_new_data_test_Sets.csv")

#### Labels

In [8]:
metrics = {}
metrics[1] = {}

forum_test = load_predict_testset("../Ablation_Studies/forum/label/test_label_preproc.csv", model_path="classif/final_label_model")
metrics[1]["GloHBCD Orig - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/test_label_preproc.csv", model_path="classif/final_label_fuzzy_model_ooD")
metrics[1]["GloHBCD Winner - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("Nonsmoking_addition/ns_label_test_set.csv", model_path="classif/final_label_model")
metrics[1]["Nonsmoking Orig - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("Nonsmoking_addition/ns_label_test_set.csv", model_path="classif/final_label_fuzzy_model_ooD")
metrics[1]["Nonsmoking Winner - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/optifast_label_preproc.csv", model_path="classif/final_label_model")
metrics[1]["Optifast Orig - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/optifast_label_preproc.csv", model_path="classif/final_label_fuzzy_model_ooD")
metrics[1]["Optifast Winner - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/virtualcoach_label_preproc.csv", model_path="classif/final_label_model")
metrics[1]["Virtualcoach Orig - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/virtualcoach_label_preproc.csv", model_path="classif/final_label_fuzzy_model_ooD")
metrics[1]["Virtualcoach Winner - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/synthetic_label_preproc.csv", model_path="classif/final_label_model")
metrics[1]["Synthetic Orig - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/synthetic_label_preproc.csv", model_path="classif/final_label_fuzzy_model_ooD")
metrics[1]["Synthetic Winner - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/WoZ_label_preproc.csv", model_path="classif/final_label_model")
metrics[1]["WoZ Orig - label"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/label/WoZ_label_preproc.csv", model_path="classif/final_label_fuzzy_model_ooD")
metrics[1]["WoZ Winner - label"] = compute_test_metrics(forum_test, None)

Using custom data configuration default-32b27a53db4c4331
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-32b27a53db4c4331/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-32b27a53db4c4331/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-84a7adbd3a5b54ca.arrow
loading configuration file classif/final_label_model/config.json
Model config BertConfig {
  "_name_or_path": "classif/final_label_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_c

Using custom data configuration default-32b27a53db4c4331
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-32b27a53db4c4331/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-4dbcc69c91c16d9d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f3e5e2f46ca24636.arrow
loading configuration file classif/final_label_model/config.json
Model config BertConfig {
  "_name_or_path": "classif/final_label_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_c

Using custom data configuration default-4dbcc69c91c16d9d
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-4dbcc69c91c16d9d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-4dbcc69c91c16d9d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f3e5e2f46ca24636.arrow
loading configuration file classif/final_label_fuzzy_model_ooD/config.json
Model config BertConfig {
  "_name_or_path": "classif/final_label_fuzzy_model_ooD",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_ty

In [9]:
metrics_lab = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()}, 
                        orient='columns').transpose()

#metrics_df = pd.concat([metrics_df, metrics_lab])
#metrics_df.tail()

In [10]:
metrics_lab.to_csv("metrics/Label_adding_new_data_test_Sets.csv")

In [11]:
metrics = {}
metrics[1] = {}

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/test_sublabel_preproc.csv", model_path="classif/final_sublabel_model")
metrics[1]["GloHBCD Orig - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/test_sublabel_preproc.csv", model_path="classif/final_sublabel_fuzzy_model_ooD")
metrics[1]["GloHBCD Winner - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("Nonsmoking_addition/ns_sublabel_test_set.csv", model_path="classif/final_sublabel_model")
metrics[1]["Nonsmoking Orig - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("Nonsmoking_addition/ns_sublabel_test_set.csv", model_path="classif/final_sublabel_fuzzy_model_ooD")
metrics[1]["Nonsmoking Winner - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/optifast_sublabel_preproc.csv", model_path="classif/final_sublabel_model")
metrics[1]["Optifast Orig - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/optifast_sublabel_preproc.csv", model_path="classif/final_sublabel_fuzzy_model_ooD")
metrics[1]["Optifast Winner - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/virtualcoach_sublabel_preproc.csv", model_path="classif/final_sublabel_model")
metrics[1]["Virtualcoach Orig - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/virtualcoach_sublabel_preproc.csv", model_path="classif/final_sublabel_fuzzy_model_ooD")
metrics[1]["Virtualcoach Winner - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/synthetic_sublabel_preproc.csv", model_path="classif/final_sublabel_model")
metrics[1]["Synthetic Orig - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/synthetic_sublabel_preproc.csv", model_path="classif/final_sublabel_fuzzy_model_ooD")
metrics[1]["Synthetic Winner - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/WoZ_sublabel_preproc.csv", model_path="classif/final_sublabel_model")
metrics[1]["WoZ Orig - sublabel"] = compute_test_metrics(forum_test, None)

forum_test = load_predict_testset("../Ablation_Studies/forum/sublabel/WoZ_sublabel_preproc.csv", model_path="classif/final_sublabel_fuzzy_model_ooD")
metrics[1]["WoZ Winner - sublabel"] = compute_test_metrics(forum_test, None)

Using custom data configuration default-4bcdf5d906130417
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-4bcdf5d906130417/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-4bcdf5d906130417/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-037c62e99eb7ee6d.arrow
loading configuration file classif/final_sublabel_model/config.json
Model config BertConfig {
  "_name_or_path": "classif/final_sublabel_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "

Using custom data configuration default-4bcdf5d906130417
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-4bcdf5d906130417/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

loading configuration file classif/final_sublabel_model/config.json
Model config BertConfig {
  "_name_or_path": "classif/final_sublabel_model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

loading weig

Using custom data configuration default-505b13c249afb61c
Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-505b13c249afb61c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-505b13c249afb61c/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-fea5e4de696f89aa.arrow
loading configuration file classif/final_sublabel_fuzzy_model_ooD/config.json
Model config BertConfig {
  "_name_or_path": "classif/final_sublabel_fuzzy_model_ooD",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "positio

In [12]:
metrics_sublab = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                            for i in metrics.keys() 
                            for j in metrics[i].keys()}, 
                        orient='columns').transpose()

#metrics_df = pd.concat([metrics_df, metrics_lab])
#metrics_df.tail()

In [13]:
metrics_sublab.to_csv("metrics/Sublabel_adding_new_data_test_Sets.csv")

#### checking winning test set label distributions

In [56]:
valence = pd.read_csv("valence_predicted_enhanced_set.csv")
valence_orig = pd.read_csv("../Ablation_Studies/forum/valence/test_valence_preproc.csv")
valence_ns = pd.read_csv("../Ablation_Studies/forum/valence/nonsmoking_valence_preproc.csv")
label = pd.read_csv("label_predicted_enhanced_set.csv")
label_orig = pd.read_csv("../Ablation_Studies/forum/label/test_label_preproc.csv")
label_ns = pd.read_csv("../Ablation_Studies/forum/label/nonsmoking_label_preproc.csv")
sublabel = pd.read_csv("sublabel_predicted_enhanced_set.csv")
sublabel_orig = pd.read_csv("../Ablation_Studies/forum/sublabel/test_sublabel_preproc.csv")
sublabel_ns = pd.read_csv("../Ablation_Studies/forum/sublabel/nonsmoking_sublabel_preproc.csv")

In [57]:
print(valence["labels"].value_counts(normalize=True))
print(valence_orig["labels"].value_counts(normalize=True))
print(valence_ns["labels"].value_counts(normalize=True))

1    0.726368
0    0.273632
Name: labels, dtype: float64
1    0.69697
0    0.30303
Name: labels, dtype: float64
1    0.734139
0    0.265861
Name: labels, dtype: float64


In [58]:
print(label["labels"].value_counts(normalize=True))
print(label_orig["labels"].value_counts(normalize=True))
print(label_ns["labels"].value_counts(normalize=True))

0    0.731434
1    0.188199
2    0.080366
Name: labels, dtype: float64
0    0.648009
1    0.256189
2    0.095802
Name: labels, dtype: float64
0    0.658610
1    0.232628
2    0.108761
Name: labels, dtype: float64


In [59]:
print(sublabel["labels"].value_counts(normalize=True))
print(sublabel_orig["labels"].value_counts(normalize=True))
print(sublabel_ns["labels"].value_counts(normalize=True))

0    0.800
2    0.080
1    0.076
3    0.044
Name: labels, dtype: float64
0    0.691275
1    0.154362
2    0.093960
3    0.060403
Name: labels, dtype: float64
0    0.704128
2    0.149083
1    0.100917
3    0.045872
Name: labels, dtype: float64


# Calculating amount of samples added per Confidence Threshold

In [ ]:
dataset_1 = load_dataset("csv", data_files="mot_05.csv", split="train")
dataset_2 = load_dataset("csv", data_files="mot_07.csv", split="train")
dataset_3 = load_dataset("csv", data_files="mot_095.csv", split="train")
dataset_4 = load_dataset("csv", data_files="new_data_both.csv", split="train")
dats = [dataset_1,dataset_2,dataset_3,dataset_4]

FN = [0.5, 0.7, 0.95, 0]
vals = [0.5, 0.7, 0.95]

In [19]:
cond = []
length = []
label_num = 2

model = AutoModelForSequenceClassification.from_pretrained("classif/final_valence_model", num_labels=label_num, ignore_mismatched_sizes=True)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
for index, dat in enumerate(dats):
    label_0_scores = []
    label_1_scores = []
    for out in pipe(KeyDataset(dat, "split"), batch_size=8, truncation="only_first"):
        label_0_scores.append(out[0]['score'])
        label_1_scores.append(out[1]['score'])
    pred_train_df = pd.DataFrame({"split": dat[:]['split'], "0":label_0_scores, "1": label_1_scores})
    predict_1 = pd.concat([_1,a_1]).sample(frac=1)
    for jindex, val in enumerate(vals):
            _1 = pred_train_df[pred_train_df["0"] > val].copy()
            a_1 = pred_train_df[pred_train_df["1"] > val].copy()
            _1["labels"] = 0
            a_1["labels"] = 1
            predict_1 = pd.concat([_1,a_1]).sample(frac=1)
            length.append(len(predict_1))
            cond.append(str(FN[index]) + " " + str(val))    
lengths = pd.DataFrame({'cond':cond, 'length':length})
lengths.to_csv("metrics/val_count_datapoints.csv")

/opt/conda/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


In [22]:
cond = []
length = []
label_num = 3

model = AutoModelForSequenceClassification.from_pretrained("classif/final_label_model", num_labels=label_num, ignore_mismatched_sizes=True)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
for index, dat in enumerate(dats):
    label_0_scores = []
    label_1_scores = []
    label_2_scores = []
    for out in pipe(KeyDataset(dat, "split"), batch_size=8, truncation="only_first"):
        label_0_scores.append(out[0]['score'])
        label_1_scores.append(out[1]['score'])
        label_2_scores.append(out[2]['score'])
    pred_train_df = pd.DataFrame({"split": dat[:]['split'], "0":label_0_scores, "1": label_1_scores, "2": label_2_scores})
    for jindex, val in enumerate(vals):
            _1 = pred_train_df[pred_train_df["0"] > val].copy()
            a_1 = pred_train_df[pred_train_df["1"] > val].copy()
            d_1 = pred_train_df[pred_train_df["2"] > val].copy()
            _1["labels"] = 0
            a_1["labels"] = 1
            d_1["labels"] = 2
            predict_1 = pd.concat([_1,a_1, d_1]).sample(frac=1)
            length.append(len(predict_1))
            cond.append(str(FN[index]) + " " + str(val))    
lengths = pd.DataFrame({'cond':cond, 'length':length})
lengths.to_csv("metrics/label_count_datapoints.csv")

/opt/conda/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


In [24]:
cond = []
length = []
label_num = 4

model = AutoModelForSequenceClassification.from_pretrained("classif/final_sublabel_model", num_labels=label_num, ignore_mismatched_sizes=True)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
for index, dat in enumerate(dats):
    label_0_scores = []
    label_1_scores = []
    label_2_scores = []
    label_3_scores = []
    for out in pipe(KeyDataset(dat, "split"), batch_size=8, truncation="only_first"):
        label_0_scores.append(out[0]['score'])
        label_1_scores.append(out[1]['score'])
        label_2_scores.append(out[2]['score'])
        label_3_scores.append(out[3]['score'])
    pred_train_df = pd.DataFrame({"split": dat[:]['split'], "0":label_0_scores, "1": label_1_scores, "2":label_2_scores, "3": label_3_scores})
    for jindex, val in enumerate(vals):
            _1 = pred_train_df[pred_train_df["0"] > val].copy()
            a_1 = pred_train_df[pred_train_df["1"] > val].copy()
            d_1 = pred_train_df[pred_train_df["2"] > val].copy()
            n_1 = pred_train_df[pred_train_df["3"] > val].copy()
            _1["labels"] = 0
            a_1["labels"] = 1
            d_1["labels"] = 2
            n_1["labels"] = 3
            predict_1 = pd.concat([_1,a_1, d_1, n_1]).sample(frac=1)
            length.append(len(predict_1))
            cond.append(str(FN[index]) + " " + str(val))    
lengths = pd.DataFrame({'cond':cond, 'length':length})
lengths.to_csv("metrics/sublabel_count_datapoints.csv")

/opt/conda/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:89: UserWarning: `return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
  warnings.warn(


In [12]:
metrics = {}
    for i in range(1):
        print(i)
        model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
        trainer = Trainer(model=model, args=training_args, train_dataset=forum_trains[i], eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
        trainer.train()
        metrics[i] = trainer.evaluate()
        dataset_1 = load_dataset("csv", data_files="mot_05.csv", split="train")
        dataset_2 = load_dataset("csv", data_files="mot_07.csv", split="train")
        dataset_3 = load_dataset("csv", data_files="mot_095.csv", split="train")
        dataset_4 = load_dataset("csv", data_files="new_data_both.csv", split="train")
        pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
        label_0_scores = []
        label_1_scores = []
        #label_2_scores = []
        #label_3_scores = []
        for out in pipe(KeyDataset(dataset_1, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
           # label_2_scores.append(out[2]['score'])
           # label_3_scores.append(out[3]['score'])
        pred_train_df_1 = pd.DataFrame({"split": dataset_1[:]['split'], "0":label_0_scores, "1": label_1_scores#, "2": label_2_scores, "3": label_3_scores 
                                       })
        label_0_scores = []
        label_1_scores = []
       # label_2_scores = []
       # label_3_scores = []
        for out in pipe(KeyDataset(dataset_2, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
            #label_2_scores.append(out[2]['score'])
            #label_3_scores.append(out[3]['score'])
        pred_train_df_2 = pd.DataFrame({"split": dataset_2[:]['split'], "0":label_0_scores, "1": label_1_scores#, "2": label_2_scores, "3": label_3_scores 
                                       })
        label_0_scores = []
        label_1_scores = []
        #label_2_scores = []
        #label_3_scores = []
        for out in pipe(KeyDataset(dataset_3, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
           # label_2_scores.append(out[2]['score'])
           # label_3_scores.append(out[3]['score'])
        pred_train_df_3 = pd.DataFrame({"split": dataset_3[:]['split'], "0":label_0_scores, "1": label_1_scores#, "2": label_2_scores, "3": label_3_scores  
                                       })
        '''label_0_scores = []
        label_1_scores = []
        label_2_scores = []
        label_3_scores = []
        for out in pipe(KeyDataset(dataset_4, "split"), batch_size=8, truncation="only_first"):
            label_0_scores.append(out[0]['score'])
            label_1_scores.append(out[1]['score'])
            label_2_scores.append(out[2]['score'])
            label_3_scores.append(out[3]['score'])
        pred_train_df_4 = pd.DataFrame({"split": dataset_4[:]['split'], "0":label_0_scores, "1": label_1_scores, "2": label_2_scores, "3": label_3_scores  })'''
        for conf_score in conf_scores:
            print(conf_score)
            _1 = pred_train_df_1[pred_train_df_1["0"] > conf_score].copy()
            a_1 = pred_train_df_1[pred_train_df_1["1"] > conf_score].copy()
            #d_1 = pred_train_df_1[pred_train_df_1["2"] > conf_score].copy()
            #n_1 = pred_train_df_1[pred_train_df_1["3"] > conf_score].copy()
            _1["labels"] = 0
            a_1["labels"] = 1
           # d_1["labels"] = 2
           # n_1["labels"] = 3
            predict_1 = pd.concat([_1,a_1#, d_1, n_1
                                  ]).sample(frac=1)
            predict_1.rename(columns={"split":"Sentence"}, inplace=True)
            predict_1.to_csv("enhanced_test_1.csv")
            enhanced_pred_1 = load_tokenize_enhanced_set("enhanced_test_1.csv")
            enhanced_1 = concatenate_datasets([forum_trains[i], enhanced_pred_1])
            enhanced_1 = enhanced_1.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_1, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["FN 05 " + str(conf_score)] = trainer.evaluate()
            trainer.save_model("classif/valence_FN_05_"+str(conf_score)+"_model")
            _2 = pred_train_df_2[pred_train_df_2["0"] > conf_score].copy()
            a_2 = pred_train_df_2[pred_train_df_2["1"] > conf_score].copy()
            #d_2 = pred_train_df_2[pred_train_df_2["2"] > conf_score].copy()
            #n_2 = pred_train_df_2[pred_train_df_2["3"] > conf_score].copy()
            _2["labels"] = 0
            a_2["labels"] = 1
           # d_2["labels"] = 2
           # n_2["labels"] = 3
            predict_2 = pd.concat([_2,a_2#, d_2, n_2
                                  ]).sample(frac=1)
            predict_2.rename(columns={"split":"Sentence"}, inplace=True)
            predict_2.to_csv("enhanced_test_2.csv")
            enhanced_pred_2 = load_tokenize_enhanced_set("enhanced_test_2.csv")
            enhanced_2 = concatenate_datasets([forum_trains[i], enhanced_pred_2])
            enhanced_2 = enhanced_2.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_2, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["FN 07 " + str(conf_score)] = trainer.evaluate()
            trainer.save_model("classif/valence_FN_07_"+str(conf_score)+"_model")
            _3 = pred_train_df_3[pred_train_df_3["0"] > conf_score].copy()
            a_3 = pred_train_df_3[pred_train_df_3["1"] > conf_score].copy()
           # d_3 = pred_train_df_3[pred_train_df_3["2"] > conf_score].copy()
           # n_3 = pred_train_df_3[pred_train_df_3["3"] > conf_score].copy()
            _3["labels"] = 0
            a_3["labels"] = 1
           # d_3["labels"] = 2
           # n_3["labels"] = 3
            predict_3 = pd.concat([_3,a_3#, d_3, n_3
                                  ]).sample(frac=1)
            predict_3.rename(columns={"split":"Sentence"}, inplace=True)
            predict_3.to_csv("enhanced_test_3.csv")
            enhanced_pred_3 = load_tokenize_enhanced_set("enhanced_test_3.csv")
            enhanced_3 = concatenate_datasets([forum_trains[i], enhanced_pred_3])
            enhanced_3 = enhanced_3.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_3, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["FN 095 " + str(conf_score)] = trainer.evaluate()
            '''_4 = pred_train_df_4[pred_train_df_4["0"] > conf_score].copy()
            a_4 = pred_train_df_4[pred_train_df_4["1"] > conf_score].copy()
            d_4 = pred_train_df_4[pred_train_df_4["2"] > conf_score].copy()
            n_4 = pred_train_df_4[pred_train_df_4["3"] > conf_score].copy()
            _4["labels"] = 0
            a_4["labels"] = 1
            d_4["labels"] = 2
            n_4["labels"] = 3
            predict_4 = pd.concat([_4,a_4, d_4, n_4]).sample(frac=1)
            predict_4.rename(columns={"split":"Sentence"}, inplace=True)
            predict_4.to_csv("enhanced_test_4.csv")
            enhanced_pred_4 = load_tokenize_enhanced_set("enhanced_test_4.csv")
            enhanced_4 = concatenate_datasets([forum_trains[i], enhanced_pred_4])
            enhanced_4 = enhanced_4.shuffle()
            model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=label_num)
            trainer = Trainer(model=model, args=training_args, train_dataset=enhanced_4, eval_dataset=forum_vals[i], compute_metrics=compute_metrics)
            trainer.train()
            metrics[i]["no FN " + str(conf_score)] = trainer.evaluate()'''
            trainer.save_model("classif/valence_FN_095_"+str(conf_score)+"_model")
            metrics_df = pd.DataFrame.from_dict({(i,j): metrics[i][j]
                               for i in metrics.keys() 
                               for j in metrics[i].keys()},
                           orient='columns').transpose()
        #metrics_df.head()
            metrics_df.to_csv("metrics/valence_FN_mot_test_sets_metrics_average_none.csv")

(3333, 12)

In [13]:
predict.shape

(992, 4)